In [1]:
from collections import defaultdict


def read_input(input_str: str, *types):
    row = input_str.split()
    return [type_(element) for type_, element in zip(types, row)]


def row_by_row_input(count: int, row_parser):
    return [
        row_parser(input())
        for _ in range(count)
    ]


class Node:
    def __init__(self, parent: int, cost: int, name: str):
        self.parent, self.cost, self.name = parent, cost, name

    def __repr__(self):
        return f"({self.name}, p={self.parent}, c={self.cost})"


class Task:
    def __init__(self, names: list, nodes: list):
        self.nodes = {
            ind + 1: node
            for ind, node in enumerate(nodes)
        }
        self.name_to_id = {
            name: 1 << ind 
            for ind, name in enumerate(set(names))
        }

    def vertex_to_cost(self, vertex: int):
        return self.nodes[vertex].cost

    def vertex_to_actions(self, vertex: int):
        return self.name_to_id[self.nodes[vertex].name]


class Tree:
    def __init__(self, task: Task):
        self.sum_costs = defaultdict(int)
        self.sons = defaultdict(list)
        self.actions = defaultdict(int)
        self._read_(task)

    def _traverse_(self, vertex: int, post_effect, leaf_process):
        if len(self.sons[vertex]) == 0:
            leaf_process(vertex)
            return
        for son in self.sons[vertex]:
            self._traverse_(son, post_effect, leaf_process)
        post_effect(vertex)

    def _union_actions_(self, vertex_idx: int, task: Task):
        self.actions[vertex_idx] = task.vertex_to_actions(vertex_idx)
        for son in self.sons[vertex_idx]:
            self.actions[vertex_idx] |= task.vertex_to_actions(son)

    def _compute_sum_(self, vertex_idx: int, task: Task):
        self.sum_costs[vertex_idx] = task.vertex_to_cost(vertex_idx)
        for son in self.sons[vertex_idx]:
            self.sum_costs[vertex_idx] += task.vertex_to_cost(son)

    def _compute_costs_(self, task: Task):
        self._traverse_(self.root, lambda x: self._compute_sum_(x, task), lambda x: self._compute_sum_(x, task))

    def _compute_actions_masks_(self, task: Task):
        self._traverse_(self.root, lambda x: self._union_actions_(x, task), lambda x: self._union_actions_(x, task))

    def _read_(self, task: Task):
        for ind, node in task.nodes.items():
            if node.parent:
                self.sons[node.parent].append(ind)
            else:
                self.root = ind
        self._compute_actions_masks_(task)
        self._compute_costs_(task)

    def find_cheapest(self, mask: int):
        return min([
            self.sum_costs[vertex]
            for vertex, action in self.actions.items()
                if (action & mask) == mask
        ])


def read_node(input_str: str):
    return Node(*read_input(input_str, int, int, str))


def input4() -> Task:
    n, k = read_input(input(), int, int)
    names = row_by_row_input(k, str)
    return Task(names=names, nodes=row_by_row_input(n, read_node))


def solve(task: Task):
    tree = Tree(task)
    count_nodes = len(task.name_to_id)
    print(tree.find_cheapest((1 << count_nodes) - 1))


if __name__ == "__main__":
    solve(input4())


5 2
A
B
0 1 A
1 2 A
1 2 B
1 1 B
4 2 A
3
